<div style="color:red;background-color:black">
Diamond Light Source

<h1 style="color:red;background-color:antiquewhite"> Linux Introduction: Job Control</h1>  

©2000-20 Chris Seddon 
</div>

In this tutorial we are going to investigate job contol.  Unfortunately, Jupyter notebook can't handle some of the job control commands, so you will need to type all the commands in a terminal window.

Linux provides the "ps" command to see all the processes running on a machine:  
- ps -ef  

but here we are only concerned with the processes we create in our current shell.  So open a terminal and create a subshell:

- bash  

before you type the commands below.

## 1
Let's start by investigating forground and background jobs.

A foreground job is one where the shell waits for the command to complete, before it will accept another command.  In Jupyter notebook things are slightly different - notebook indicates the shell is waiting by printing a '*' in the [ ] at the side of the cell.

Lets run the "sleep" command for 10 seconds and watch the '*' appear for 10 seconds; the shell is waiting for the sleep command to complete.

In [7]:
sleep 10
echo "sleep completed"

[1]+  Done                    sleep 10
sleep completed


## 2
Now let's run the "sleep" command again, but this time in the background.  Now the shell won't wait for the "sleep" to complete and will move on to the next command immediately - and you won't see a '*' either.

To run a coomand in the background add an & at the end of the command:

In [1]:
sleep 10 &
echo "sleep completed"

[1] 62423
sleep completed


## 3
If we use a long running command, the shell will wait for a long time.  I'm going to use the "xeyes" program in this example.
Let's use the "xeyes" command to investigate job control:

In [ ]:
xeyes

In [4]:
xeyes&

[2] 62428


When you execute the script above, the "xeyes" are created, but you don't get your prompt back.  This is because you have executed the "xeyes" command in what we call the *foreground*.  All commands are executed in the foreground unless you specify the command should run in the background (see below).

Now kill off the "xeyes" using the cross in the top right corner of the title bar.  Now you get your prompt.  What is happening?

When you run a command in the foreground, the bash shell waits for the command to complete before the shell issues a prompt ready for the next command.  

FOREGROUND: the shell waits  

The alternative is to run a command in the background:  


- xeyes &

The & tells the shell to run the xeyes command in the backgound. 

BACKGROUND: the shell does NOT wait

Create a couple more xeyes in the background:

- xeyes&
- xeyes&

Run the "jobs" command to see what commands are running in the background:

- jobs

You get output like:  

[1]   Running                 xeyes &  
[2]-  Running                 xeyes &  
[3]+  Running                 xeyes &  


The numbers are the local process ids.  
We can now do some job control.  We can *suspend*, *restart* and *terminate* jobs, but the job control commands are different for foreground and background jobs.  In fact to interact with foreground jobs you can't use any commands because you haven't got your prompt back.  Instead you use control sequences.

Let's start with the background jobs.  
To suspend a background job type:  
- kill -TSTP %2  

Notice that one of the xeyes is incapacitated.  The command "kill" originally killed the job, but nowadays the command has been extended to perform suspend, restart and terminate.  In fact kill sends a signal to the job.  Look in the manual pages to see all the possible signals:  
- man 7 signal

Now restart the job:  
- kill -CONT %2  

The job is restarted.

Now terminate the job:  
- kill -TERM %2  

The job is terminated.

Jobs can be transferred from background to forground:  
- fg  %1  

This takes job 1 into the foreground.

Let's suspend this job to get our prompt back:  
- Contrl-Z
(hold the control key down while pressing Z).  

Restart the job in the background:  
- bg %1

Move the job to the foreground:  
- fg %1

Terminate the foreground job:  
- Control-C

The local job ids can be omitted from the *fg* and *bg*.  In that case the command will apply to the most recent job.  You can also replace the local job ids with the process ids as displayed by the "ps" command.

Finally, you will often see people use the special KILL signal.  When you send any of the signals above, you are sending the signals from the bash shell, via the Linux kernel, to the target command.  Target commands should obey the signal, but they are allowed to ignore the signal.  However if the command crashes or gets stuck in a loop it will ignore terminate signals, so you won't be able to "kill" the command.  
To get around this problem a special signal is used (KILL).  This signal is not delivered to the process; the kernel just terminates the process abruptly:  
- kill -KILL %3

Finally, let's summarise the above commands:

| command | result
| --- | ---
| *xeyes* | run in foreground
| xeyes& | run in background
| kill -TERM %1 | send terminate signal
| kill -TSTP %1 | send suspend (temporary stop) signal
| kill -CONT %1 | send restart (continue) signal
| fg %1 | restart process in foreground
| bg %1 | restart process in background
| Control-Z | send suspend signal to foreground process
| Control-C | send terminate signal to foreground process
| jobs | list background jobs
| kill -KILL %1 | send unstoppable kill signal